In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.linear_model import LinearRegression
import statsmodels.formula.api as smf

In [12]:
df_well_being_raw = pd.read_excel("data/london-ward-well-being-probability-scores.xls", sheet_name="Dashboard", skiprows=12)
df_grocery_raw = pd.read_csv('data/year_osward_grocery.csv')

In [19]:
df_well_being_raw

,Borough name,Ward Code,Ward name,Unnamed: 3,Unnamed: 4,2009,2010,2011,2012,2013,Unnamed: 10,Unnamed: 11,See full data set (e.g. numbers/rates),Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19
0,City of London,E09000001,City of London,NaN,NaN,3.868138,2.584469,2.159794,1.046416,1.420837,NaN,-0.611825,NaN,NaN,NaN,-1.283670,-0.424675,-1.113377,0.374420,-0.611825
1,Barking and Dagenham,E05000026,Abbey,NaN,NaN,3.695918,3.854677,2.949578,1.542211,0.699065,NaN,-0.749213,NaN,NaN,NaN,0.158758,-0.905099,-1.407367,-0.843146,NaN
2,Barking and Dagenham,E05000027,Alibon,NaN,NaN,-10.931432,-10.714408,-11.112796,-11.310643,-8.902220,NaN,0.507303,NaN,NaN,NaN,0.217024,-0.398388,-0.197847,2.408423,NaN
3,Barking and Dagenham,E05000028,Becontree,NaN,NaN,-5.054122,-4.774793,-5.492461,-6.776579,-10.425686,NaN,-1.342891,NaN,NaN,NaN,0.279329,-0.717668,-1.284118,-3.649107,NaN
4,Barking and Dagenham,E05000029,Chadwell Heath,NaN,NaN,-9.132546,-9.747185,-9.333469,-11.121355,-8.457127,NaN,0.168855,NaN,NaN,NaN,-0.614638,0.413716,-1.787886,2.664228,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
654,Sutton,E09000029,SUTTON,NaN,NaN,4.823202,4.813937,4.880437,5.037866,4.053126,NaN,-0.192519,NaN,NaN,NaN,-0.009265,0.066500,0.157429,-0.984739,NaN
655,Tower Hamlets,E09000030,TOWER HAMLETS,NaN,NaN,-6.282362,-6.346922,-6.096303,-5.404874,-1.792618,NaN,1.122436,NaN,NaN,NaN,-0.064559,0.250618,0.691429,3.612257,NaN
656,Waltham Forest,E09000031,WALTHAM FOREST,NaN,NaN,-1.818464,-1.614183,-0.974500,-1.223081,0.230698,NaN,0.512291,NaN,NaN,NaN,0.204281,0.639683,-0.248581,1.453779,NaN
657,Wandsworth,E09000032,WANDSWORTH,NaN,NaN,-2.496451,-2.157883,-2.097240,-1.853591,-0.429535,NaN,0.516729,NaN,NaN,NaN,0.338568,0.060643,0.243649,1.424056,NaN


In [20]:
df_well = df_well_being_raw[["Ward Code", 2013]]
df_well.columns = ['area_id', 'well_being']
df_well = df_well.drop(0)
df_well = df_well.dropna()

In [21]:
df_well

,area_id,well_being
1,E05000026,0.699065
2,E05000027,-8.902220
3,E05000028,-10.425686
4,E05000029,-8.457127
5,E05000030,-7.604599
...,...,...
654,E09000029,4.053126
655,E09000030,-1.792618
656,E09000031,0.230698
657,E09000032,-0.429535


In [6]:
#df_grocery = df_grocery_raw[["area_id", "fat", "saturate", "salt", "sugar", "protein", "carb", "fibre", "alcohol"]]
df_grocery = df_grocery_raw[["area_id", "fat", "saturate", "salt", "sugar", "protein", "carb", "fibre", "alcohol", "energy_fat", "energy_saturate","energy_sugar", 
                             "energy_protein", "energy_carb", "energy_fibre", "energy_alcohol", "energy_tot", "h_nutrients_calories_norm",
                            'f_beer', 'f_dairy', 'f_eggs', 'f_fats_oils', 'f_fish', 'f_fruit_veg', 'f_grains', 'f_meat_red', 'f_poultry',
                             'f_readymade', 'f_sauces', 'f_soft_drinks', 'f_spirits', 'f_sweets', 'f_tea_coffee', 'f_water', 'f_wine']]

nutrients = ["fat", "protein", "carb", "fibre", "alcohol"]
df_grocery_percentage = df_grocery.copy()
for i, nutrient in enumerate (nutrients):
    df_grocery_percentage["percentage_%s"%nutrient] = df_grocery_percentage["energy_%s"%nutrient] *100 / df_grocery["energy_tot"]
    


In [7]:
df_grocery_percentage

,area_id,fat,saturate,salt,sugar,protein,carb,fibre,alcohol,energy_fat,...,f_spirits,f_sweets,f_tea_coffee,f_water,f_wine,percentage_fat,percentage_protein,percentage_carb,percentage_fibre,percentage_alcohol
0,E05000026,9.488797,3.693721,0.583240,10.966213,4.977560,19.381951,1.564721,0.198172,85.399176,...,0.004656,0.181254,0.005945,0.042942,0.007917,45.640000,10.640657,41.433292,1.544686,0.741364
1,E05000027,9.733634,3.565913,0.568184,10.514427,5.211694,18.950348,1.581960,0.209917,87.602703,...,0.005180,0.173620,0.005572,0.041258,0.007355,46.439549,11.051200,40.183491,1.546797,0.778963
2,E05000028,9.216310,3.613582,0.610536,10.690272,5.192412,19.662048,1.572323,0.225425,82.946787,...,0.003657,0.179634,0.006076,0.043609,0.009281,44.389378,11.114978,42.088964,1.562220,0.844460
3,E05000029,9.700137,3.961264,0.533180,12.938606,4.718184,20.084734,1.550344,0.200380,87.301229,...,0.004576,0.216788,0.006578,0.046902,0.007344,45.759214,9.892203,42.109906,1.503467,0.735209
4,E05000030,9.381808,3.614663,0.566784,11.332898,5.307003,19.581403,1.607947,0.168952,84.436271,...,0.002538,0.189401,0.006638,0.039688,0.009738,44.880395,11.283322,41.632398,1.575264,0.628621
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633,E05011485,9.059381,3.504138,0.543892,11.155665,5.059010,19.018689,1.556461,0.251503,81.534433,...,0.004312,0.176239,0.006308,0.039785,0.018416,44.678745,11.088822,41.686985,1.580729,0.964718
634,E05011486,8.819808,3.367769,0.586726,10.804966,5.174936,19.238456,1.574568,0.234103,79.378269,...,0.003801,0.166791,0.006518,0.045984,0.015791,43.711512,11.398800,42.376434,1.610854,0.902399
635,E05011487,9.156464,3.434680,0.554677,11.401595,5.132303,19.398455,1.665961,0.230386,82.408176,...,0.004320,0.178311,0.006296,0.040283,0.012097,44.489578,11.083074,41.890458,1.666242,0.870648
636,E05011488,9.557516,3.706401,0.582808,12.093208,5.119245,20.472076,1.606728,0.177276,86.017640,...,0.003668,0.194153,0.007404,0.048004,0.008467,44.667084,10.633250,42.522809,1.532470,0.644387


In [8]:
df = df_well.merge(df_grocery_percentage, on='area_id')

In [9]:
df

,area_id,well_being,fat,saturate,salt,sugar,protein,carb,fibre,alcohol,...,f_spirits,f_sweets,f_tea_coffee,f_water,f_wine,percentage_fat,percentage_protein,percentage_carb,percentage_fibre,percentage_alcohol
0,E05000026,6.118185,9.488797,3.693721,0.583240,10.966213,4.977560,19.381951,1.564721,0.198172,...,0.004656,0.181254,0.005945,0.042942,0.007917,45.640000,10.640657,41.433292,1.544686,0.741364
1,E05000027,-4.726708,9.733634,3.565913,0.568184,10.514427,5.211694,18.950348,1.581960,0.209917,...,0.005180,0.173620,0.005572,0.041258,0.007355,46.439549,11.051200,40.183491,1.546797,0.778963
2,E05000028,-4.813479,9.216310,3.613582,0.610536,10.690272,5.192412,19.662048,1.572323,0.225425,...,0.003657,0.179634,0.006076,0.043609,0.009281,44.389378,11.114978,42.088964,1.562220,0.844460
3,E05000029,-4.813479,9.700137,3.961264,0.533180,12.938606,4.718184,20.084734,1.550344,0.200380,...,0.004576,0.216788,0.006578,0.046902,0.007344,45.759214,9.892203,42.109906,1.503467,0.735209
4,E05000030,-4.726708,9.381808,3.614663,0.566784,11.332898,5.307003,19.581403,1.607947,0.168952,...,0.002538,0.189401,0.006638,0.039688,0.009738,44.880395,11.283322,41.632398,1.575264,0.628621
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478,E05000645,16.097385,9.011954,3.558849,0.584727,9.403091,5.499603,16.729870,1.609476,0.292334,...,0.003565,0.144103,0.006161,0.026018,0.021485,46.319170,12.562921,38.216582,1.732698,1.168630
479,E05000646,16.097385,8.810350,3.379945,0.604999,9.350598,5.584789,17.240961,1.640990,0.209072,...,0.003073,0.143007,0.005728,0.030805,0.014232,45.269606,12.753748,39.372456,1.768656,0.835535
480,E05000647,16.097385,8.642757,3.398100,0.584833,9.446028,5.321004,17.579308,1.680360,0.299310,...,0.003656,0.143096,0.004823,0.026675,0.022156,44.537008,12.186522,40.261319,1.815528,1.199623
481,E05000648,15.507184,8.919674,3.512147,0.549463,9.623928,5.259262,17.940815,1.611090,0.195706,...,0.002644,0.152942,0.006401,0.037393,0.015903,45.227374,11.852083,40.430772,1.717957,0.771814


In [10]:
spearman_nutrients = {}
spearman_weight = {}
spearman_energy = {}

for nutrient in nutrients:
    spearman_nutrients[nutrient] = stats.spearmanr(df["percentage_%s"%nutrient],df['well_being'])
    spearman_weight[nutrient] = stats.spearmanr(df["%s"%nutrient],df['well_being'])
    spearman_energy[nutrient] = stats.spearmanr(df["energy_%s"%nutrient],df['well_being'])
    
spearman_entropy = stats.spearmanr(df["h_nutrients_calories_norm"],df['well_being'])

In [11]:
# Fit linear regression model to nutrients
mod = smf.ols(formula='well_being ~  fat + sugar + alcohol + carb + saturate + protein + fibre', data=df)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:             well_being   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     1.467
Date:                Fri, 11 Dec 2020   Prob (F-statistic):              0.177
Time:                        14:09:28   Log-Likelihood:                -1784.5
No. Observations:                 483   AIC:                             3585.
Df Residuals:                     475   BIC:                             3618.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -7.3516     19.021     -0.387      0.6

In [12]:
# Fit logistic regression model to foods
mod = smf.ols(formula='well_being ~  f_beer + f_dairy + f_eggs + f_fats_oils+f_fish+f_fruit_veg+f_grains+f_meat_red+f_poultry+f_readymade + f_sauces+f_soft_drinks+f_spirits+f_sweets+f_tea_coffee+f_water+f_wine', data=df)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:             well_being   R-squared:                       0.101
Model:                            OLS   Adj. R-squared:                  0.070
Method:                 Least Squares   F-statistic:                     3.257
Date:                Fri, 11 Dec 2020   Prob (F-statistic):           2.18e-05
Time:                        14:09:29   Log-Likelihood:                -1764.0
No. Observations:                 483   AIC:                             3562.
Df Residuals:                     466   BIC:                             3633.
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept       -28.1847     42.416     -0.664

In [34]:
### Let's build a healhty diet according to https://en.wikipedia.org/wiki/Healthy_diet

foods = dict(zip(["fruit_veg", "grains", "dairy", "meat_red", "poultry", "eggs", "fish"], [4.5, 0.75, 3, 1.5/7, 1.25/7, 0.375/7, 1/7]))
sum_values = sum(foods.values())

for food in foods.keys():
    foods[food] = foods[food]/sum_values

In [35]:
foods

{'fruit_veg': 0.5090909090909091,
 'grains': 0.08484848484848485,
 'dairy': 0.3393939393939394,
 'meat_red': 0.024242424242424242,
 'poultry': 0.020202020202020204,
 'eggs': 0.006060606060606061,
 'fish': 0.01616161616161616}

In [37]:
df_healthy = df.copy()
df_healthy.loc[:,"score"] =0
for food in foods:
    df_healthy["score"] += (df_healthy["f_%s"%food] - foods[food])**2
    

In [38]:
df_healthy

,area_id,well_being,fat,saturate,salt,sugar,protein,carb,fibre,alcohol,...,f_water,f_wine,percentage_fat,percentage_saturate,percentage_sugar,percentage_protein,percentage_carb,percentage_fibre,percentage_alcohol,score
0,E05000026,6.118185,9.488797,3.693721,0.583240,10.966213,4.977560,19.381951,1.564721,0.198172,...,0.042942,0.007917,45.640000,17.766364,23.442754,10.640657,41.433292,1.544686,0.741364,0.125700
1,E05000027,-4.726708,9.733634,3.565913,0.568184,10.514427,5.211694,18.950348,1.581960,0.209917,...,0.041258,0.007355,46.439549,17.013110,22.295443,11.051200,40.183491,1.546797,0.778963,0.130958
2,E05000028,-4.813479,9.216310,3.613582,0.610536,10.690272,5.192412,19.662048,1.572323,0.225425,...,0.043609,0.009281,44.389378,17.404437,22.883805,11.114978,42.088964,1.562220,0.844460,0.138758
3,E05000029,-4.813479,9.700137,3.961264,0.533180,12.938606,4.718184,20.084734,1.550344,0.200380,...,0.046902,0.007344,45.759214,18.686781,27.127245,9.892203,42.109906,1.503467,0.735209,0.152978
4,E05000030,-4.726708,9.381808,3.614663,0.566784,11.332898,5.307003,19.581403,1.607947,0.168952,...,0.039688,0.009738,44.880395,17.291712,24.095093,11.283322,41.632398,1.575264,0.628621,0.139938
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478,E05000645,16.097385,9.011954,3.558849,0.584727,9.403091,5.499603,16.729870,1.609476,0.292334,...,0.026018,0.021485,46.319170,18.291587,21.479785,12.562921,38.216582,1.732698,1.168630,0.102913
479,E05000646,16.097385,8.810350,3.379945,0.604999,9.350598,5.584789,17.240961,1.640990,0.209072,...,0.030805,0.014232,45.269606,17.366936,21.353566,12.753748,39.372456,1.768656,0.835535,0.102567
480,E05000647,16.097385,8.642757,3.398100,0.584833,9.446028,5.321004,17.579308,1.680360,0.299310,...,0.026675,0.022156,44.537008,17.510757,21.633932,12.186522,40.261319,1.815528,1.199623,0.105918
481,E05000648,15.507184,8.919674,3.512147,0.549463,9.623928,5.259262,17.940815,1.611090,0.195706,...,0.037393,0.015903,45.227374,17.808405,21.688136,11.852083,40.430772,1.717957,0.771814,0.097920


In [39]:
stats.spearmanr(df_healthy["score"],df_healthy['well_being'])

SpearmanrResult(correlation=0.09502979534896164, pvalue=0.036814974492699666)